In [59]:
import pandas as pd
import feather

In [60]:
df = feather.read_dataframe('consolidated.feather')

In [61]:
df.head()

,product_name,page_link,price,description,image_link,brand
0,Straight-cut Jacket,https://www2.hm.com/en_us/productpage.06364550...,34.99,Straight-cut jacket in woven fabric with notch...,http://lp2.hm.com/hmgoepprod?set=source[/07/25...,H&M
1,Linen-blend Jacket,https://www2.hm.com/en_us/productpage.07554510...,59.99,"Gently fitted jacket in soft, woven fabric mad...",http://lp2.hm.com/hmgoepprod?set=source[/27/f3...,H&M
2,Double-breasted Jacket,https://www2.hm.com/en_us/productpage.07178790...,34.99,"Longer, double-breasted jacket in woven fabric...",http://lp2.hm.com/hmgoepprod?set=source[/2c/3d...,H&M
3,Linen-blend Jacket,https://www2.hm.com/en_us/productpage.07246260...,49.99,Gently fitted jacket in a linen and viscose bl...,http://lp2.hm.com/hmgoepprod?set=source[/89/66...,H&M
4,Fitted Blazer,https://www2.hm.com/en_us/productpage.05686010...,34.99,Fitted blazer in woven fabric with notched lap...,http://lp2.hm.com/hmgoepprod?set=source[/c0/a1...,H&M


In [62]:
len(df.product_name.unique()), len(df)

(17625, 33222)

In [63]:
# df['description'] = df['product_name'] + ' ' + df['description']

In [64]:
df['description'] = df['description'].astype(str)

In [65]:
df['description'] = df['description'].apply(lambda x: x.lower().replace('description',''))

In [66]:
df.reset_index(inplace=True)
df.head()

,index,product_name,page_link,price,description,image_link,brand
0,0,Straight-cut Jacket,https://www2.hm.com/en_us/productpage.06364550...,34.99,straight-cut jacket in woven fabric with notch...,http://lp2.hm.com/hmgoepprod?set=source[/07/25...,H&M
1,1,Linen-blend Jacket,https://www2.hm.com/en_us/productpage.07554510...,59.99,"gently fitted jacket in soft, woven fabric mad...",http://lp2.hm.com/hmgoepprod?set=source[/27/f3...,H&M
2,2,Double-breasted Jacket,https://www2.hm.com/en_us/productpage.07178790...,34.99,"longer, double-breasted jacket in woven fabric...",http://lp2.hm.com/hmgoepprod?set=source[/2c/3d...,H&M
3,3,Linen-blend Jacket,https://www2.hm.com/en_us/productpage.07246260...,49.99,gently fitted jacket in a linen and viscose bl...,http://lp2.hm.com/hmgoepprod?set=source[/89/66...,H&M
4,4,Fitted Blazer,https://www2.hm.com/en_us/productpage.05686010...,34.99,fitted blazer in woven fabric with notched lap...,http://lp2.hm.com/hmgoepprod?set=source[/c0/a1...,H&M


In [67]:
# !pip install -U gensim
# ! sudo rm -f /etc/boto.cfg
# !pip install google-compute-engine

In [68]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Text processing

In [69]:
from bs4 import BeautifulSoup

def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
df['description']= df['description'].apply(cleanText)

In [70]:
from sklearn.feature_extraction import stop_words
import string
def clean_text(text):
    """
    Tokenize text and return a non-unique list of tokenized words
    found in the text. Normalize to lowercase, strip punctuation,
    remove stop words, drop words of length < 3, strip digits.
    """
    stops = list(stop_words.ENGLISH_STOP_WORDS)
    text = text.lower()
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)  # delete stuff but leave at least a space to avoid clumping together
    words = nopunct.split(" ")
    words = [w for w in words if (len(w) > 2 and (w not in stops))]  # ignore a, an, to, at, be, ...
    words = ' '.join(words)


    return words

In [71]:
df['description'] = df['description'].apply(lambda x: clean_text(x))

Tokenize

In [72]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [73]:
df_tagged = df.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['description']), tags=[r['index']]), axis=1)


In [74]:
df_tagged.head()

0    ([straight, cut, jacket, woven, fabric, notche...
1    ([gently, fitted, jacket, soft, woven, fabric,...
2    ([longer, double, breasted, jacket, woven, fab...
3    ([gently, fitted, jacket, linen, viscose, blen...
4    ([fitted, blazer, woven, fabric, notched, lape...
dtype: object

In [29]:
! unzip -a glove.6B.100d.txt.zip

Archive:  glove.6B.100d.txt.zip
  inflating: glove.6B.100d.txt       [binary]


model 1 - doc2vec model from scratch

In [87]:
model = Doc2Vec(vector_size=100, min_count=2, workers=6)
%time model.build_vocab(df_tagged)

CPU times: user 664 ms, sys: 0 ns, total: 664 ms
Wall time: 663 ms


model 2 - Initialize doc2vec with word2vec embeddings 

In [75]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

(400000, 100)

In [77]:
from gensim.models import Word2Vec
word2vec = Word2Vec(size=100)
%time word2vec.build_vocab([word for text in df_tagged for word in text])

CPU times: user 316 ms, sys: 0 ns, total: 316 ms
Wall time: 315 ms


In [81]:
%time word2vec.intersect_word2vec_format("glove.6B.100d.txt.word2vec",lockf=1.0, binary=False, encoding='utf8', unicode_errors='strict')

CPU times: user 32.7 s, sys: 820 ms, total: 33.5 s
Wall time: 31.9 s


Train models <br>
model 1 - model <br>
model 2 - model_pretrained <br>

In [88]:
%time model.train(df_tagged, total_examples=model.corpus_count, epochs=30)

CPU times: user 2min 20s, sys: 49.7 s, total: 3min 10s
Wall time: 1min 33s


In [89]:
model_pretrained = Doc2Vec(vector_size=100, min_count=2, workers=6)
model_pretrained.build_vocab(df_tagged)

In [90]:
model_pretrained.wv = word2vec.wv
%time model_pretrained.train(df_tagged, total_examples=model.corpus_count, epochs=30)

CPU times: user 2min 19s, sys: 51.2 s, total: 3min 10s
Wall time: 1min 33s


Find similar vectors from both the models 

In [104]:
inferred_vector = model.infer_vector(df_tagged[0].words, steps=200)
model.docvecs.most_similar([inferred_vector], topn=5)

[(2242, 0.6167574524879456),
 (10, 0.5978948473930359),
 (33051, 0.5920794606208801),
 (23731, 0.5832704305648804),
 (18, 0.5827090740203857)]

In [93]:
inferred_vector = model_pretrained.infer_vector(df_tagged[0].words, steps=20)
model_pretrained.docvecs.most_similar([inferred_vector], topn=5)

[(31599, 0.8587937951087952),
 (32491, 0.8429571390151978),
 (33061, 0.8381904363632202),
 (32336, 0.8298468589782715),
 (28822, 0.8279505968093872)]

In [55]:
list(df[df['index'].isin(['0','30947','30779','32364','29831','29850'])]['page_link'])

['https://www2.hm.com/en_us/productpage.0636455005.html',
 'https://shop.mango.com/us/women/jackets-blazers/belt-structured-blazer_41095821.html?c=08&n=1&s=prendas_she.familia;16,304',
 'https://shop.mango.com/us/women/jackets-blazers/belt-structured-blazer_41095821.html?c=08&n=1&s=prendas_she.familia;17',
 'https://shop.mango.com/us/women/jackets-blazers/structured-linen-jacket_43065745.html?c=81&n=1&s=prendas_she.familia;17',
 'https://shop.mango.com/us/women/jumpsuits-long/polka-dot-long-jumpsuit_41085749.html?c=56&n=1&s=prendas_she.familia;34',
 'https://shop.mango.com/us/women/coats-coats/unstructured-wool-blend-coat_41090769.html?c=99&n=1&s=prendas_she.familia;15']

In [56]:
inferred_vector = model_pretrained.infer_vector(df_tagged[0].words, steps=20)
model_pretrained.docvecs.most_similar([inferred_vector], topn=5)

[(26701, 0.7871068716049194),
 (7523, 0.785920262336731),
 (23116, 0.7799015045166016),
 (26707, 0.7683223485946655),
 (6341, 0.7644155621528625)]

In [57]:
list(df[df['index'].isin(['0','26701','7523','23116','26707','6341'])]['page_link'])

['https://www2.hm.com/en_us/productpage.0636455005.html',
 'https://www.macys.com//shop/product/calvin-klein-textured-one-button-blazer?ID=7950280&CategoryID=188851',
 'https://www.macys.com//shop/product/rachel-rachel-roy-jacquard-midi-skirt-created-for-macys?ID=3097258&CategoryID=188851',
 'http://us.topshop.com/en/tsus/product/clothing-70483/cable-knit-crop-jumper-8491108',
 'http://us.topshop.com/en/tsus/product/clothing-70483/textured-boucle-jacket-8319734',
 'http://us.topshop.com/en/tsus/product/clothing-70483/bonded-belted-jacket-8319684']

#####################################        THE END         #############################################

In [58]:
from gensim.test.utils import get_tmpfile
fname_model = get_tmpfile("muimui_doc2vec_model")
model.save(fname_model)
fname_model_pretrained = get_tmpfile("muimui_doc2vec_model_pretrained")
model_pretrained.save(fname_model_pretrained)

In [127]:
model_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
model_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [130]:
# ! pip install testfixtures

In [131]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

In [133]:
def get_vectors(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [134]:
index, doc_vec = get_vectors(new_model, df_tagged)

In [137]:
# ! pip install annoy

In [138]:
from gensim.similarities.index import AnnoyIndexer

In [140]:
annoy_index = AnnoyIndexer(model_dbow, num_trees=50)